# Example of Use

In [29]:
import re                                 # Für Textbereinigung (Kraftwerksnamen)
import pandas as pd
import geopandas as gpd                   # Für räumliche Operationen (z. B. Nearest-Join: kombiniert Daten aus zwei räumlichen Datensätzen auf der Grundlage ihrer Nähe zueinander)

import powerplantmatching as pm
from powerplantmatching.cleaning import set_column_name, get_config, aggregate_units  # Hilfsfunktionen des Tools


In [30]:
heatcap_path = r"C:\Users\delic\Desktop\DataUnits_CHP.xlsx"           # Excel-Datei mit thermischer Leistung (CHP_MaxHeat)
ppmlist_path  = r"C:\Users\delic\powerplantmatching\powerplants.csv"  # bestehende Kraftwerksliste aus PPM

#PPM: id	  Name	Fueltype Technology	Set	          Country	  Capacity	Efficiency DateIn	           DateOut	   lat	    lon
#EWL: unit_it unit	fuel	 deftech	operationmode countrycode max_power	AvgEff	   effective_startdate valid_until latitude	longitude CHP_MaxHeat

heatcap = pd.read_excel(heatcap_path)     # eigene Daten einlesen
ppm_list = pd.read_csv(ppmlist_path, sep=",")  # PPM-Kraftwerksdaten einlesen


In [31]:
heatcap.columns = heatcap.columns.str.strip()
ppm_list.columns  = ppm_list.columns.str.strip()

# Koordinaten in numerisches Format umwandeln und auf 2 Nachkommastellen runden (≈ 300 m Toleranz)
for df in (heatcap, ppm_list):
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce").round(2)
    df["lon"] = pd.to_numeric(df["lon"], errors="coerce").round(2)


In [32]:
heatcap["CHP_MaxHeat"] = pd.to_numeric(heatcap.get("CHP_MaxHeat"), errors="coerce") # Thermische Leistung numerisch umwandeln

In [33]:
# Relevante Spalten definieren, fehlende mit pd.NA auffüllen
match_cols = [
"Name","Fueltype","Technology","Set","DateIn","Country","City","Capacity","Efficiency","lat","lon","CHP_MaxHeat"
]

#"DateOut", (fragliche Datenqualität, in der DataUnits immer 1970)

heatcap_match = heatcap.reindex(columns=match_cols, fill_value=pd.NA).copy()
ppm_list_match = ppm_list.reindex(columns=match_cols, fill_value=pd.NA).copy()

# 'Set'-Spalte (z. B. „PP“ oder „CHP“) ergänzen, falls leer
for df in (heatcap_match, ppm_list_match):
    df["Set"] = df["Set"].fillna("PP")

In [34]:
# Funktion zur einheitlichen Bereinigung von Strings
def _clean_strings(df, cols):
    for c in cols:
        df[c] = (
            df[c].astype(str)
                 .str.strip()
                 .replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA, "None": pd.NA})
        )

# Funktion zur Vereinheitlichung der Kraftwerksnamen
def clean_name(s: str) -> str:
    if pd.isna(s):
        return s
    s = (s.replace("ÃŸ", "ß")               # Zeichencodierung reparieren
           .replace("Gross", "Groß"))       # Schreibweise vereinheitlichen
    s = re.sub(r"\b(Block|Unit|Bk|Bl|B)\s*[0-9IVX]+\b", "", s, flags=re.I)  # Blocknummern entfernen
    s = re.sub(r"\b(Kraftwerk|Power\s*Plant|HKW|KW)\b", "", s, flags=re.I) # generische Begriffe entfernen
    s = re.sub(r"\s+", " ", s).strip().title()  # Doppelte Leerzeichen entfernen + Case angleichen
    return s

_clean_strings(heatcap_match, match_cols)
_clean_strings(ppm_list_match,  match_cols)

# Bereinigte Namen auf beide Tabellen anwenden
heatcap_match["Name"] = heatcap_match["Name"].map(clean_name)
ppm_list_match["Name"]  = ppm_list_match["Name"].map(clean_name)

In [35]:
# Datenquellen mit Namen versehen (für konkretes Matching)
# Hier lag vorher eine der Haupftfehlerquellen: PPM erwartet pro Spalte genau eine Überschrift
heatcap_match = set_column_name(heatcap_match, name="EWL-DataUnits")
ppm_list_match = set_column_name(ppm_list_match, name="PPMLIST")

In [36]:
# Matching-Konfiguration festlegen
config = get_config()
config["target_columns"] = [            # Spalten, die im Endergebnis enthalten sein sollen
"Name","Fueltype","Technology","Set","Country","City","Capacity","Efficiency","DateIn","lat","lon","CHP_MaxHeat"
]

config["EWL-DataUnits"] = {"reliability_score": 7}   # Vertrauenswürdigkeit (für Auswahl im Ergebnis)
config["PPMLIST"]   = {"reliability_score": 6}

In [37]:
# Matching ausführen: welche Einträge gehören zusammen?
dfs    = [heatcap_match, ppm_list_match]
labels = ["EWL-DataUnits", "PPMLIST"]

# Matching mit Hilfe von powerplantmatching
intersection = pm.matching.combine_multiple_datasets(
    dfs, labels=labels, config=config,
)

In [38]:
# Alle EWL-DataUnits-Anlagen, die noch keinen Partner in PPMLIST haben
missing = intersection[("Name", "PPMLIST")].isna()
leftover = intersection.loc[missing].copy()

if len(leftover):
    # a) linke Geometrien = EWL-DataUnits-Koordinaten
    g_left = gpd.GeoDataFrame(
        leftover,
        geometry=gpd.points_from_xy(
            leftover[("lon", "EWL-DataUnits")],
            leftover[("lat", "EWL-DataUnits")]
        ),
        crs="EPSG:4326"
    ).to_crs(3857)

    # b) rechte Geometrien = alle PPMLIST-Koordinaten
    base = ppm_list_match.copy()
    g_right = gpd.GeoDataFrame(
        base,
        geometry=gpd.points_from_xy(base.lon, base.lat),
        crs="EPSG:4326"
    ).to_crs(3857)[["Name", "lat", "lon", "geometry"]]

    # c) Führe einen Nearest-Join durch (max. 1 km Entfernung)
    hits = gpd.sjoin_nearest(
        g_left, g_right,
        how="left",
        max_distance=1000,          # 1 km
        distance_col="d_m"
    )
    hits = hits[hits["Name"].notna()].copy()  # Nur Treffer übernehmen

    # d) Ergänze in intersection die PPMLIST-Werte aus dem spatial match
    for idx, row in hits.iterrows():
        intersection.loc[idx, ("Name", "PPMLIST")] = row["Name"]
        intersection.loc[idx, ("lat",  "PPMLIST")] = row["lat"]
        intersection.loc[idx, ("lon",  "PPMLIST")] = row["lon"]

# Reduktion nach spatial Join erneut durchführen: Kombiniert Attribute aus zwei räumlichen Datensätzen (Feature-Klassen oder Layer) auf Grundlage ihrer räumlichen Beziehungen.
combined = intersection.powerplant.reduce_matched_dataframe(config=config)
print(f"number of final matched plans: {len(combined)}")  # Ausgabe der Zahl final gematchter Kraftwerke

number of final matched plans: 182


In [40]:
display(intersection)  # zeigt das vollständige Ergebnis mit beiden Quellen nebeneinander


EWL-DataUnits                                      Name                  \
                                          EWL-DataUnits         PPMLIST   
0                                          Farge Bremen           Farge   
1                           Staudinger5 Großkrotzenburg      Staudinger   
2                                               Zolling         Zolling   
3                                    Gkhblock1 Hannover             Gkh   
4                                   Hafenblock 6 Bremen           Hafen   
..                                                  ...             ...   
177                                      Bohrhügel Suhl       Bohrhugel   
178                                      Unterbreizbach  Unterbreizbach   
179                                Wiesengrund Eisenach     Wiesengrund   
180                          Emslandgt 1 Gt 2 Dt Lingen              Gt   
181            Südgud C 200 Dt 1 Gt 1 Gt 2 Ludwigshafen              Dt   

EWL-DataUnits      Fueltype                      Technology                 \
              EWL-DataUnits      PPMLIST      EWL-DataUnits        PPMLIST   
0                      COAL    Hard Coal      COAL_ST_f2000  Steam Turbine   
1                      COAL    Hard Coal      COAL_ST_b1999  Steam Turbine   
2                      COAL    Hard Coal  COAL_ST_EXC_b1989  Steam Turbine   
3                      COAL    Hard Coal  COAL_ST_BKP_b1983  Steam Turbine   
4                      COAL    Hard Coal  COAL_ST_EXC_b1983  Steam Turbine   
..                      ...          ...                ...            ...   
177                 NAT_GAS  Natural Gas   GAS_GT_CH2_f2000  Steam Turbine   
178                 NAT_GAS  Natural Gas   GAS_ST_EXC_b1969  Steam Turbine   
179                 NAT_GAS  Natural Gas   GAS_ST_BKP_b1999  Steam Turbine   
180                 NAT_GAS  Natural Gas   GAS_CC_EXC_f2000           CCGT   
181                 NAT_GAS  Natural Gas   GAS_CC_BKP_f2000           <NA>   

EWL-DataUnits           Set               Country           ...    Efficiency  \
              EWL-DataUnits PPMLIST EWL-DataUnits  PPMLIST  ... EWL-DataUnits   
0                        PP      PP       Germany  Germany  ...          0.46   
1                        PP     CHP       Germany  Germany  ...         0.403   
2                       CHP     CHP       Germany  Germany  ...         0.363   
3                       CHP     CHP       Germany  Germany  ...         0.245   
4                       CHP     CHP       Germany  Germany  ...         0.329   
..                      ...     ...           ...      ...  ...           ...   
177                     CHP     CHP       Germany  Germany  ...          0.35   
178                     CHP     CHP       Germany  Germany  ...           0.3   
179                     CHP     CHP       Germany  Germany  ...          0.27   
180                     CHP     CHP       Germany  Germany  ...          0.56   
181                     CHP     CHP       Germany  Germany  ...          0.51   

EWL-DataUnits                            DateIn                   lat          \
                          PPMLIST EWL-DataUnits PPMLIST EWL-DataUnits PPMLIST   
0                            <NA>          1970  1969.0          53.2    53.2   
1                          0.3597          1992  1965.0         50.09   50.09   
2                            <NA>          1986  1985.0         48.45   48.46   
3                            <NA>          1989  1989.0         52.41   52.42   
4                            <NA>          1979  1968.0         53.13   53.12   
..                            ...           ...     ...           ...     ...   
177            0.3794999999999999          1995  1995.0         50.64   50.63   
178                        0.3795          1970  1995.0          50.8   50.81   
179                        0.3773          1993  1993.0         50.99   50.98   
180                        0.5445          2010  1973.0         